In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/leonardo/Desktop/neo4j-connector-apache-spark_2.11-4.0.0.jar pyspark-shell'



from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import TimestampType, StringType, StructType, StructField
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.streaming import StreamingContext
from operator import attrgetter
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.streaming import StreamingContext


from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import monotonically_increasing_id
from py2neo import Graph


graph = Graph(host="localhost",password="unodos1234")
#cypher = graph.cypher()



from pyspark.ml import PipelineModel
#Se carga el modelo que previamente se entreno para luego usarlo en la evaluacion de los streams de data
StreamPipeline = PipelineModel.load("Spipeline_2")


#Se define el esquema de los datos que se leeran en el stream
spark = SparkSession.builder.master("local").appName("Avila-stream").config("spark.some.config.option","some-value").getOrCreate()
schema = "F1 DOUBLE, F2 DOUBLE, F3 DOUBLE, F4 DOUBLE, F5 DOUBLE, F6 DOUBLE, F7 DOUBLE, F8 DOUBLE, F9 DOUBLE, F10 DOUBLE, AuthorNum DOUBLE"

#Definir la variable que va leer los archivos guardados en la carpeta /test/ 
streamingDF = (
  spark
    .readStream
    .schema(schema)
    .option("header","true")
    .option("maxFilesPerTrigger", 1)
    .csv("test/")
)
#Declarar el evaluador que se usara para evaluar las predicciones del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="AuthorNum",predictionCol="prediction", metricName="accuracy")


def get_cluster(df):
    cols=df.columns
    cols.remove("AuthorNum")
    assembler = VectorAssembler(inputCols=cols,outputCol="features")
    data = df
    data = assembler.transform(data)
    
    kmeans = KMeans().setK(12).setSeed(1)
    model = kmeans.fit(data)
    df_cluster = model.transform(data)
    n = df_cluster.drop("features")
    #n.show()
    n.write.format("org.neo4j.spark.DataSource").option("url",
    "bolt://localhost:7687").option("authentication.type",
    "basic").option("authentication.basic.username","neo4j").option("authentication.basic.password","unodos1234").option("labels",
    ":Paginas").save()
    
    graph.run("MATCH (a:Paginas) MATCH (b:Paginas) WHERE a.prediction = b.prediction MERGE (a)-[r:link]->(b)")

#Función que se encarga de transformar cada bache y calcular las predicciones de este.
def train_df(df,epoch_id):
    
    print("----WORKING ON BATCH----")
    print(".................")
    print("# REGISTROS EN BATCH:",df.count(),", Atributos:",len(df.columns))
    get_cluster(df)
    prediction = StreamPipeline.transform(df)
    prediction.show()
    dt_accuracy = evaluator.evaluate(prediction)
    print("----TEST STREAMING RESULTS----")
    print("----BATCH PREDICTIONS---")
    print("Accuracy of RandomForest is = {}" .format(dt_accuracy))

query = streamingDF.writeStream.foreachBatch(train_df).start()   
query.awaitTermination()
    




----WORKING ON BATCH----
.................
# REGISTROS EN BATCH: 1981 , Atributos: 11


StreamingQueryException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/home/leonardo/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 2442, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/home/leonardo/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 207, in call
    raise e
  File "/home/leonardo/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 204, in call
    self.func(DataFrame(jdf, self.sql_ctx), batch_id)
  File "<ipython-input-1-44fd6459e7e3>", line 77, in train_df
    get_cluster(df)
  File "<ipython-input-1-44fd6459e7e3>", line 67, in get_cluster
    ":Paginas").save()
  File "/home/leonardo/anaconda3/lib/python3.7/site-packages/pyspark/sql/readwriter.py", line 825, in save
    self._jwrite.save()
  File "/home/leonardo/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1305, in __call__
    answer, self.gateway_cli